##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Text classification with movie reviews

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/tutorials/keras/basic_text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/r2.0rc/site/en/tutorials/keras/basic_text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

En este cuaderno se clasifican las evaluaciones de usuarios a películas como *positivas* o *negativas*. Este es un ejeplo de una clasificación *binaria* o de dos clases, un problema ampliamente aplicable en el aprendizaje automático.

Se usará el conjunto de datos de [IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) que contiene el texto de 50,000 evaluaciones de películas de la Internet Movie DataBase (IMBD). Este conjunto se divide en 25,000 evaluaciones para el entrenamiento y 25,000 para la prueba. Los conjuntos de entrenamiento y prueba estan *balanceados*, lo que significa que ambos contienen la misma cantidad de evaluaciones positivas y negativas.

Este cuaderno usa [tf.keras](https://www.tensorflow.org/guide/keras), una API de alto nivel para entrenar modelos en TensorFlow. Para un tutorial más avanzado en clasificación de texto usando `tf.keras`, ve a [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/).

In [ ]:
# keras.datasets.imdb is broken in 1.13 and 1.14, by np 1.16.3
!pip install tf_nightly

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

## Descargar el conjunto de datos IMDB

El conjunto de datos IMDB viene incluido con TensorFlow. Ya ha sido preprocesado de tal forma que las evaluaciones (secuencias de palabras) se han convertido a secuencias de enteros, donde cada entero representa una palabra específica en un diccionario.

El siguiente código descarga el conjunto de datos IMDB a tu máquina (o usa una copia del caché si ya lo descargaste):

In [ ]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

El argumento `num_words=10000` mantiene las 10,000 palabras más frecuentes que ocurren en el conjunto de datos de entrenamiento. Las palabras raras se descartan para mantener manejabla el tamaño de los datos.

## Exploración de los datos

Nos tomaremos un momento para entender el formato de los datos. el conjunto de datos esta preprocesado: cada ejemplo en el arreglo de enteros representa a las palabras de la evaluación de las películas. Cada etiqueta es un valor entero ya se de 0 o 1, donde 0 es una evaluación negativa y 1 es una evaluación positiva.

In [ ]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

El texto de las evaluaciones se ha convertido a enteros, donde cada entero representa una palabra específica en un diccionario. Así es como se ve la primera evaluación:

In [ ]:
print(train_data[0])

Las evaluaciones de las películas pueden tener distinta cantidad de palabras. En el código de abajo se muestra el número de palabras en la primera y segunda evaluación. 
Ya que las entradas a las redes neuronales deben tener el mismo tamaño, tendremos que resolver esto.

In [ ]:
len(train_data[0]), len(train_data[1])

### Convertir los enteros de nuevo a palabras

Puede ser útil convertir los enteros nuevamente a texto. En el siguiente código creamos una función de ayuda para buscar las palabras en un diccionario que contiene los enteros para mapearlos al texto.


In [ ]:
# Un diccionario que mapea las palabras a un índice entero
word_index = imdb.get_word_index()

# Los primeros índices están reservados
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

Ahora podemos usar la función `decode_review` para desplegar el texto para la primera evaluación:

In [ ]:
decode_review(train_data[0])

## Preparar los datos

Los arreglos con las evaluaciones que contienen los enteros deben de convertirse a tensores antes de usarse para introducirse a las redes neuronales. Esta conversión se puede hacer de dos formas:

* Convertir los arreglos en vectores con 0s y 1s que indican que palabras de un diccionario ocurren en una evaluación, similar a lo que se conoce como one-hot enoding. Por ejemplo, la secuencia [3, 5] se convertería un vector con 10,000 elementos que estaría lleno de ceros, excepto en los índices 3 y 5 que se refiere a las palabras que ocurren. Entonces, esto sería la primera capa de la red neuronal. Esta forma requiere bastante memoria, ya que requiere una matriz de `num_words * num_reviews`.

* Alternativamente, podemos hacer padding (o rellenar) los arreglos de forma que todos tengan la misma longitud, después crear un tensor de enteros con una forma `max_length * num_reviews`. Podemos usar una capa embebida capaz de manejar esta forma como la primer capa de la red.

En este tutorial, usaremos la segunda forma.


Ya que las evaluaciones de las películas deben tener la misma longitud, usaremos la función [pad_sequences](https://keras.io/preprocessing/sequence/#pad_sequences) para estandarizar las longitudes:

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

Vamos a ver la longitud de los ejemplos ahora:

In [ ]:
len(train_data[0]), len(train_data[1])

E inspeccionar la primera evaluación (ahora rellenada):

In [ ]:
print(train_data[0])

## Construir el modelo

La red neuronal se crea al apilar capas, lo que requiero dos decisiones de arquitectura principales:

* ¿Cuántas capas se usarán en el modelo?
* ¿Cuántas *hidden units* (unidades ocultas) se usarán en cada capa?

En este ejemplo, los datos de entrada consisten en un arreglo con índices de palabras. Las etiquetas para la predicción son ya sea 0 o 1. Vamos a construir un modelo para este problema:

In [ ]:
# la forma de la entrada es el contador del vocabulario usado para las evaluaciones de las pelícuas (10,000 palabras)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Las capas estan apiladas de forma secuencial para construir al clasificador:

1. La primera capa es una capa `Embebida`. Esta capa toma el vocabulario de enteros codificados y busca el vector embebido para cada indice de palabra. Estos datos se aprende conforme el modelo se entrena. Los vector agregan una dimensión al arreglo de salida. La dimensión resultante es:>

1. The layers are stacked sequentially to build the classifier:`(batch, sequence, embedding)`.
2. A una capa llamada `GlobalAveragePooling1D`  regresa un vector de salida con un tamaño fijo para caad ejemplo, promediando sobre la dimensión de la secuencia. Esto permite al modelo la capacidad de manipular la longitud de la variable de entrada, de la manera más fácil posible.
3. Este vector de tamaño fijo se conduce a través de una capa completamente conectada (`Dense`) con 16 unidades escondidas.
4. La última capa es una capa densamiente conectada con solo un nodo de salida. Por medio de la función de activación `sigmoid`, este valor fluctúa entre 0 y 1, representando una probabilidad o nivel de confianza.

### Unidades ocultas

El modelo tiene dos capas intermedias "ocultas", entre las capas de entrada y de salida. el número de salidas (unidades, nodos, o neuronas) es la dimensión del espacio de representación para cada capa. En otras palabras, la cantidad de libertad de la red esta permitida cuando se aprende una representación interna.

Si un modelo tiene más unidades ocultas (un espacio de representación de dimensión mas alta), u otro con mas capas. entonces la red puede aprender representaciones más complejas. Sin emabrgo, esto provoca que la red requiere más recursos computacionales y podría llegar a aprender patrones no deseados que mejoran el desempeño en los datos de entrenamiento pero no en los datos de prueba, a esto se le conoce como *overfitting* y se verá mas adelante.

### Optimizador y función de pérdida

Un modelo necesita una función de pérdida y un optimizador para el entrenamiento. Ya que es un problema de clasificación binaria, y el modelo entrega una probabilidad como salida (una capa con un solo nodo con una función de activación sigmoide(, usaremos la función de pérdida `binary_crossentropy`.

Este no es la única decisión respecto a la función de pérdida, se puede, por ejemplo, escoger `mean_squared_error`. Pero, generalmente la `binary_crossentropy` es mejor para tratar con probabilidades, ya que mide la "distancia"entre las distribuciones de probabilidad, o en nuestro caso, entre la distribución de nuestro "ground-truth" y las predicciones.

Ahora, configuramos el modelo para usar un optimizador y una función de pérdida:

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

## Creación de un conjunto de validación

Cuando entrenamos, queremos checar la exactitud del modelo en datos que no ha visto nunca. Creamos un *conjunto de validación* al separar 10,000 ejemplos del conjunto de datos de entrenamiento original. (Porqué no solo usar el conjunto de prueba de una vez? Nuestra meta es desarrollar y ajustar nuestro modelos usando únicamente datos de entrenamiento, y después usar datos de prueba solo una vez para evaluar nuestra exactitud)


In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Entrenamiento del modelo

Entrar el modelo por 40 époas en mini-lotes de 512 ejemplos. Esto es, 40 iteraciones sobre todas las muestras de tensores `x_train` y `y_train`. Mientras se entrena, se monitorea la pérdida y exactitud en las 10,000 muestras del conjunto de validación.

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

## Evaluar el modelo

Veamos como se desempeña el modelo. Se regresan dos valores. La pérdida (un número que representa nuestro error, valores más bajos son mejores), y exactitud.

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)

Esta propuesta, que es algo ingenua, logra una exactitud de 87%. Con propuestas de arquitecturas más avanzadas, el modelo puede incrementar a cerca de 95%.

## Crear un gráfica de exactitud y pérdida a través del tiempo

`model.fit()` regresa un objeto `History` que contiene un diccionario con todo lo que sucedió durante el entrenamiento:

In [ ]:
history_dict = history.history
history_dict.keys()

Hay cuantro entradas: una para cada medida monitoreada durante el entrenamiento y validación. Esamos estos valores para graficar y comparar la pérdida y exactitud en el entrenamiento y validación:

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

En esta gráfica, los puntos representan la pérdida y la exactitud en el entrenamiento mientras que la linea sólida representa la pérdida y exactitud en la validación.

Nota que la pérdida *disminuye* en cada época durante el entrenamiento y la exactitud *incremementa* con cada época. Esto es esperado la que cuando se usa el algoritmo de optimización del descenso del grandiente se debe minimizar la cantidad deseada sobre cada iteración.

Esto no es el caso para la périda y la exactitud en la validación, ya que se ven que llegan a un pico en las 20 épocas. Este es un ejemplo de *overfitting*: el modelo funciona mejor en los datos de entremaiento que en datos que no ha visto nunca. Después de este punto, el modelo sobre-optimiza y encuentra representaciones *específicas* para los datos de entrenamiento que no *generalizan* a los datos de prueba.

Para este caso particular, podemos prevenir el *overfitting* al parar el entrenamiento después de las 20 épocas.

## Predecir una evaluación

In [ ]:
results = model.predict(test_data[[1]])
print("Probability: ", results)
print("Label predicted: ",test_labels[1])
print("Evaluación:")
decode_review(test_data[1])

Mostrar palabras del diccionario

In [ ]:
print(list(word_index.keys())[0:10])  